In [417]:
import numpy as np

# We define a, b, c, alpha, beta, gamma

a, b, c = 0.4, 0.4, 0.405
a_star, b_star, c_star = 0.25, 0.25, 0.2469
alpha, beta, gamma = 90, 90, 90

#check calculations
volume_init = 0.0648

B = np.transpose(np.array([1, 1, 0]))
g = np.transpose(np.array([0, 0, 2]))

def get_volume(a, b, c, alpha, beta, gamma):
    
    alpha, beta, gamma = np.deg2rad(alpha), np.deg2rad(beta), np.deg2rad(gamma)
    sin_alpha, cos_alpha =  np.sin(alpha), np.cos(alpha)
    sin_beta, cos_beta = np.sin(beta), np.cos(beta)
    sin_gamma, cos_gamma = np.sin(gamma), np.cos(gamma)

    volume_ = 1.0 - cos_alpha**2.0 - cos_beta**2.0 - cos_gamma**2.0 + 2.0 * cos_alpha * cos_beta * cos_gamma
    volume = a*b*c*np.sqrt(volume_)
    
    return volume

def fractional_to_cartesian_M(a, b, c, alpha, beta, gamma, volume):

    alpha, beta, gamma = np.deg2rad(alpha), np.deg2rad(beta), np.deg2rad(gamma)
    sin_alpha, cos_alpha =  np.sin(alpha), np.cos(alpha)
    sin_beta, cos_beta = np.sin(beta), np.cos(beta)
    sin_gamma, cos_gamma = np.sin(gamma), np.cos(gamma)
        
    M = np.zeros((3, 3))
    M[0, 0] = a
    M[0, 1] = b * cos_gamma
    M[1, 1] = b * sin_gamma
    M[0, 2] = c * cos_beta
    M[1, 2] = c * (cos_alpha - cos_beta * cos_gamma) / sin_gamma
    M[2, 2] = volume / (a*b*sin_gamma)
    
    return M

volume = get_volume(a, b, c, alpha, beta, gamma)

M = fractional_to_cartesian_M(a, b, c, alpha, beta, gamma, volume)
M_inv = np.linalg.inv(M)

print('Transformation Matrix M: {} '.format(M))

print('Volume: {} '.format(volume))

# Step 1. We use unit cell vectors and multiply transformation matrix N to calculate a,b,c in X,Y,Z 

a_XYZ, b_XYZ, c_XYZ = np.dot(M, [1,0,0]), np.dot(M, [0,1,0]), np.dot(M, [0,0,1])
print('----------')

print('a_XYZ: ',format(a_XYZ))
print('b_XYZ: ',format(b_XYZ))
print('c_XYZ: ',format(c_XYZ))

re_check_volume = np.dot(a_XYZ,np.cross(b_XYZ, c_XYZ))
re_check_volume2 = np.linalg.det(M)
print('Re-check volume: {}'.format(re_check_volume))
print('Re-check volume2: {}'.format(re_check_volume2))

# Step 2. We use a,b,c in X,Y,Z to calculate a_star_XYZ, b_star_XYZ, c_star_XYZ

# Reciprocal lattice vectors
a_star_XYZ = np.cross(b_XYZ, c_XYZ)/volume
b_star_XYZ = np.cross(c_XYZ, a_XYZ)/volume
c_star_XYZ = np.cross(a_XYZ, b_XYZ)/volume
print('----------')

print('a_star_XYZ: ',format(a_star_XYZ))
print('b_star_XYZ: ',format(b_star_XYZ))
print('c_star_XYZ: ',format(c_star_XYZ))

# Step 3. We use a, b, c, a_star, b_star, c_star in XYZ to calculate B and g vectors in XYA

B_XYZ = B[0] * a_XYZ + B[1]* b_XYZ + B[2] * c_XYZ
g_XYZ = g[0] * a_star_XYZ + g[1] * b_star_XYZ + g[2]* c_star_XYZ
print('----------')

print('B_XYZ: {}'.format(B_XYZ))
print('g_XYZ: {}'.format(g_XYZ))

# Step 4. We normalize vectors  and calculate x_XYZ as cross product of y and z

# g || y; B || z; x cross product y,z

z_XYZ = B_XYZ/np.linalg.norm(B_XYZ)
y_XYZ = g_XYZ/np.linalg.norm(g_XYZ)
x_XYZ = np.cross(y_XYZ, z_XYZ)

print('----------')

print('x_XYZ: ',format(x_XYZ))
print('y_XYZ: ',format(y_XYZ))
print('z_XYZ: ',format(z_XYZ))

# Step 4. We create transformation matrix from xyz -> XYZ

transform_matrix = np.array([x_XYZ, y_XYZ, z_XYZ]).T
inv_transform_matrix = np.linalg.inv(transform_matrix)

print('----------')

# Step 5. We calculate a,b,c in XYZ

print('Transformation matrix: ', inv_transform_matrix)
a_xyz = np.dot(inv_transform_matrix, a_XYZ)
b_xyz = np.dot(inv_transform_matrix, b_XYZ)
c_xyz = np.dot(inv_transform_matrix, c_XYZ)

print('----------')
print('a_xyz: {}'.format(a_xyz))
print('b_xyz: {}'.format(b_xyz))
print('c_xyz: {}'.format(c_xyz))

Transformation Matrix M: [[4.00000000e-01 2.44929360e-17 2.47990977e-17]
 [0.00000000e+00 4.00000000e-01 2.47990977e-17]
 [0.00000000e+00 0.00000000e+00 4.05000000e-01]] 
Volume: 0.06480000000000001 
----------
a_XYZ:  [0.4 0.  0. ]
b_XYZ:  [2.4492936e-17 4.0000000e-01 0.0000000e+00]
c_XYZ:  [2.47990977e-17 2.47990977e-17 4.05000000e-01]
Re-check volume: 0.06480000000000001
Re-check volume2: 0.06480000000000001
----------
a_star_XYZ:  [ 2.5000000e+00 -1.5308085e-16 -1.5308085e-16]
b_star_XYZ:  [ 0.0000000e+00  2.5000000e+00 -1.5308085e-16]
c_star_XYZ:  [0.        0.        2.4691358]
----------
B_XYZ: [0.4 0.4 0. ]
g_XYZ: [0.        0.        4.9382716]
----------
x_XYZ:  [-0.70710678  0.70710678  0.        ]
y_XYZ:  [0. 0. 1.]
z_XYZ:  [0.70710678 0.70710678 0.        ]
----------
Transformation matrix:  [[-0.70710678  0.70710678 -0.        ]
 [ 0.          0.          1.        ]
 [ 0.70710678  0.70710678  0.        ]]
----------
a_xyz: [-0.28284271  0.          0.28284271]
b_xyz: [0.

In [418]:
check_x = np.array([1, 0, 0])
check_y = np.array([0, 1, 0])
check_z = np.array([0, 0, 1])

transform_x_to_abc = np.dot(M_inv, np.dot(transform_matrix, check_x))
transform_y_to_abc = np.dot(M_inv, np.dot(transform_matrix, check_y))
transform_z_to_abc = np.dot(M_inv, np.dot(transform_matrix, check_z))

print(transform_x_to_abc)
print(transform_y_to_abc)
print(transform_z_to_abc)


[-1.76776695  1.76776695  0.        ]
[-1.5308085e-16 -1.5308085e-16  2.4691358e+00]
[1.76776695 1.76776695 0.        ]
